## DECISION TREE MODEL - 1 Parameter Work's Great

In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

numpoints= 1000000
start_time = time.time()
data = pd.read_csv('datasets/Energy/fuchs_v3_points_'+str(numpoints)+'_noise_10.csv')

features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
target = data[['Max_Proton_Energy_(MeV)']]

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Decision Tree Regressor with optimized parameters
dt = DecisionTreeRegressor(max_depth=15, min_samples_leaf=6, random_state=400)

# We'll have to fit a separate model for each target variable
models = {}
mse_errors = []
rmse_errors = []
are_errors = []
for column in target.columns:
    dt.fit(features_train, target_train[column])
    
    # Get the trained model
    models[column] = dt
    
    # Predict on the test set
    target_test_pred = models[column].predict(features_test)
    
    # Calculate the MSE
    mse_error = mean_squared_error(target_test[column], target_test_pred)
    mse_errors.append(mse_error)
    
    # Calculate the RMSE
    rmse_error = np.sqrt(mse_error)
    rmse_errors.append(rmse_error)
    
    # Calculate the ARE in percentage
    are_error = (mean_absolute_error(target_test[column], target_test_pred) / np.mean(target_test[column])) * 100
    are_errors.append(are_error)

# Print the MSE, RMSE, and ARE for each target variable
for i, column in enumerate(target.columns):
    print(f'MSE for {column}: {mse_errors[i]}')
    print(f'RMSE for {column}: {rmse_errors[i]}')
    print(f'ARE for {column}: {are_errors[i]}%')

end_time = time.time()  # record the end time

elapsed_time = end_time - start_time  # calculate elapsed time

print("Elapsed time: {} seconds".format(elapsed_time))


MSE for Max_Proton_Energy_(MeV): 0.0028949234652069144
RMSE for Max_Proton_Energy_(MeV): 0.05380449298345738
ARE for Max_Proton_Energy_(MeV): 8.908335662494972%
Elapsed time: 5.881248235702515 seconds


## RANDOM FOREST -  3 Parameter

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import time

data = pd.read_csv('datasets/Energy/fuchs_v3_points_1000000_noise_10.csv')
features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
target = data[['Max_Proton_Energy_(MeV)', 'Avg_Proton_Energy_(MeV)', 'Total_Proton_Energy_(MeV)']]

start_time = time.time()

# Split the dataset into train and test sets
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [400],
    'max_depth': [20],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['auto'],
}

# Initialize the Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(features_train, target_train)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Predict on the test set using the best model
target_test_pred = best_model.predict(features_test)

# Calculate the MSE for each target variable
mse_errors = []
rmse_errors = []
are_errors = []
for i, column in enumerate(target.columns):
    mse_error = mean_squared_error(target_test[column], target_test_pred[:, i])
    rmse_error = np.sqrt(mse_error)
    are_error = (mean_absolute_error(target_test[column], target_test_pred[:, i]) / np.mean(target_test[column])) * 100
    mse_errors.append(mse_error)
    rmse_errors.append(rmse_error)
    are_errors.append(are_error)

# Print the MSE, RMSE, and ARE for each target variable
for i, column in enumerate(target.columns):
    print(f'MSE for {column}: {mse_errors[i]}')
    print(f'RMSE for {column}: {rmse_errors[i]}')
    print(f'ARE for {column}: {are_errors[i]}%')

# Print the best hyperparameters from the grid search
print("Best Hyperparameters:", grid_search.best_params_)

end_time = time.time()  # record the end time
elapsed_time = end_time - start_time  # calculate elapsed time
print("Elapsed time: {} seconds".format(elapsed_time))


MSE for Max_Proton_Energy_(MeV): 0.002786213053761169
RMSE for Max_Proton_Energy_(MeV): 0.05278459106369177
ARE for Max_Proton_Energy_(MeV): 8.652768304749364%
MSE for Avg_Proton_Energy_(MeV): 8.475914037276833e-05
RMSE for Avg_Proton_Energy_(MeV): 0.009206472743280585
ARE for Avg_Proton_Energy_(MeV): 8.457726922635963%
MSE for Total_Proton_Energy_(MeV): 515386288338080.7
RMSE for Total_Proton_Energy_(MeV): 22702120.789434645
ARE for Total_Proton_Energy_(MeV): 8.670554005655184%
Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Elapsed time: 234.14374136924744 seconds


## Random Forest - One Parameter

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

start_time = time.time()
data = pd.read_csv('datasets/Energy/fuchs_v3_points_1000000_noise_10.csv')

# Select features and target
features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
target = data[['Max_Proton_Energy_(MeV)']]

# Split the dataset into train and test sets
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Define the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=400, max_depth=20, min_samples_split=2, min_samples_leaf=1, max_features='auto', random_state=42)

# Fit the model on the training data
rf.fit(features_train, target_train.values.ravel())

# Predict on the test set
target_test_pred = rf.predict(features_test)

# Calculate the MSE
mse_error = mean_squared_error(target_test, target_test_pred)

# Calculate the RMSE
rmse_error = np.sqrt(mse_error)

# Calculate the ARE in percentage
are_error = (mean_absolute_error(target_test, target_test_pred) / np.mean(target_test, axis=0)) * 100

# Extract the scalar value from the Series object
are_error_scalar = are_error.values.item()

# Format the ARE to display as a percentage with two decimal places
are_error_formatted = "{:.2f}%".format(are_error_scalar)

# Print the MSE, RMSE, and ARE
print(f'MSE: {mse_error}')
print(f'RMSE: {rmse_error}')
print(f'ARE: {are_error_formatted}')

end_time = time.time()  # record the end time
elapsed_time = end_time - start_time  # calculate elapsed time
print("Elapsed time: {} seconds".format(elapsed_time))



MSE: 0.0025797301145896842
RMSE: 0.050791043645407444
ARE: 8.38%
Elapsed time: 979.9962062835693 seconds


In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

start_numpoints = 5000
end_numpoints = 50000
interval = 1000

results = []

for numpoints in range(start_numpoints, end_numpoints + 1, interval):
    start_time = time.time()
    print(f"Number of Points: {numpoints}")

    data = pd.read_csv(f'datasets/Energy/fuchs_v3_points_{numpoints}_noise_10.csv')

    features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
    target = data[['Max_Proton_Energy_(MeV)']]

    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Decision Tree Regressor with optimized parameters
    dt = DecisionTreeRegressor(max_depth=15, min_samples_leaf=6, random_state=400)

    # We'll have to fit a separate model for each target variable
    models = {}
    mse_errors = []
    rmse_errors = []
    are_errors = []
    for column in target.columns:
        dt.fit(features_train, target_train[column])

        # Get the trained model
        models[column] = dt

        # Predict on the test set
        target_test_pred = models[column].predict(features_test)

        # Calculate the MSE
        mse_error = mean_squared_error(target_test[column], target_test_pred)
        mse_errors.append(mse_error)

        # Calculate the RMSE
        rmse_error = np.sqrt(mse_error)
        rmse_errors.append(rmse_error)

        # Calculate the ARE in percentage
        are_error = (mean_absolute_error(target_test[column], target_test_pred) / np.mean(target_test[column])) * 100
        are_errors.append(are_error)

    elapsed_time = time.time() - start_time  # calculate elapsed time

    # Store the results in a dictionary
    result = {
        'numpoints': numpoints,
        'MSE': mse_errors[0],
        'RMSE': rmse_errors[0],
        'ARE': are_errors[0],
        'Elapsed Time (seconds)': elapsed_time
    }

    results.append(result)

    print(f'MSE for Max_Proton_Energy_(MeV): {mse_errors[0]}')
    print(f'RMSE for Max_Proton_Energy_(MeV): {rmse_errors[0]}')
    print(f'ARE for Max_Proton_Energy_(MeV): {are_errors[0]}%')
    print("Elapsed time: {} seconds".format(elapsed_time))
    print()

# Save the results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv('results_max_proton_energy.csv', index=False)



Number of Points: 5000
MSE for Max_Proton_Energy_(MeV): 0.009658534833291956
RMSE for Max_Proton_Energy_(MeV): 0.09827784507859315
ARE for Max_Proton_Energy_(MeV): 15.250317981654263%
Elapsed time: 0.07729601860046387 seconds

Number of Points: 6000
MSE for Max_Proton_Energy_(MeV): 0.009779774130905758
RMSE for Max_Proton_Energy_(MeV): 0.09889274053693607
ARE for Max_Proton_Energy_(MeV): 14.684217592050427%
Elapsed time: 0.09435772895812988 seconds

Number of Points: 7000
MSE for Max_Proton_Energy_(MeV): 0.0070194817597692555
RMSE for Max_Proton_Energy_(MeV): 0.0837823475427208
ARE for Max_Proton_Energy_(MeV): 13.770978333273028%
Elapsed time: 0.43993186950683594 seconds

Number of Points: 8000
MSE for Max_Proton_Energy_(MeV): 0.00724435113749313
RMSE for Max_Proton_Energy_(MeV): 0.0851137541029247
ARE for Max_Proton_Energy_(MeV): 13.515904199705552%
Elapsed time: 0.09365463256835938 seconds

Number of Points: 9000
MSE for Max_Proton_Energy_(MeV): 0.006312345750914583
RMSE for Max_Prot

MSE for Max_Proton_Energy_(MeV): 0.003818631235717955
RMSE for Max_Proton_Energy_(MeV): 0.061795074526356505
ARE for Max_Proton_Energy_(MeV): 10.43094156574281%
Elapsed time: 0.4074742794036865 seconds

Number of Points: 42000
MSE for Max_Proton_Energy_(MeV): 0.004432296833054401
RMSE for Max_Proton_Energy_(MeV): 0.06657549724226175
ARE for Max_Proton_Energy_(MeV): 10.979029052542991%
Elapsed time: 0.1962277889251709 seconds

Number of Points: 43000
MSE for Max_Proton_Energy_(MeV): 0.0037991108082379497
RMSE for Max_Proton_Energy_(MeV): 0.06163692731016002
ARE for Max_Proton_Energy_(MeV): 10.078359410216985%
Elapsed time: 0.17138266563415527 seconds

Number of Points: 44000
MSE for Max_Proton_Energy_(MeV): 0.003977818262672353
RMSE for Max_Proton_Energy_(MeV): 0.06306994738123976
ARE for Max_Proton_Energy_(MeV): 10.45735293415348%
Elapsed time: 0.17608642578125 seconds

Number of Points: 45000
MSE for Max_Proton_Energy_(MeV): 0.003703694034765549
RMSE for Max_Proton_Energy_(MeV): 0.060

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time

start_numpoints = 5000
end_numpoints = 50000
interval = 1000

results = []

for numpoints in range(start_numpoints, end_numpoints + 1, interval):
    start_time = time.time()
    print(f"Number of Points: {numpoints}")

    data = pd.read_csv(f'datasets/Energy/fuchs_v3_points_{numpoints}_noise_10.csv')

    features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
    target = data[['Max_Proton_Energy_(MeV)']]

    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Define the Random Forest Regressor
    rf = RandomForestRegressor(n_estimators=400, max_depth=20, min_samples_split=2, min_samples_leaf=1, max_features='auto', random_state=42)

    # Fit the model on the training data
    rf.fit(features_train, target_train.values.ravel())

    # Predict on the test set
    target_test_pred = rf.predict(features_test)

    # Calculate the MSE
    mse_error = mean_squared_error(target_test, target_test_pred)

    # Calculate the RMSE
    rmse_error = np.sqrt(mse_error)

    # Calculate the ARE in percentage
    are_error = (mean_absolute_error(target_test, target_test_pred) / np.mean(target_test, axis=0)) * 100

    # Extract the scalar value from the Series object
    are_error_scalar = are_error.values.item()

    # Format the ARE to display as a percentage with two decimal places
    are_error_formatted = "{:.2f}%".format(are_error_scalar)

    # Store the results in a dictionary
    result = {
        'Number of Points': numpoints,
        'MSE': mse_error,
        'RMSE': rmse_error,
        'ARE': are_error_scalar,
        'Elapsed Time (seconds)': time.time() - start_time
    }

    results.append(result)

    print(f'MSE: {mse_error}')
    print(f'RMSE: {rmse_error}')
    print(f'ARE: {are_error_formatted}')

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('results.csv', index=False)


Number of Points: 5000
MSE: 0.005567399879629501
RMSE: 0.07461501108777979
ARE: 11.55%
Number of Points: 6000
MSE: 0.0045278390579760065
RMSE: 0.06728921947813042
ARE: 10.58%
Number of Points: 7000
MSE: 0.003923498309054081
RMSE: 0.06263783448566913
ARE: 10.48%
Number of Points: 8000
MSE: 0.00418953398000893
RMSE: 0.06472660952042004
ARE: 10.07%
Number of Points: 9000
MSE: 0.0034557748895463042
RMSE: 0.058785839192328486
ARE: 9.79%
Number of Points: 10000
MSE: 0.003420079868799291
RMSE: 0.05848144892869269
ARE: 9.76%
Number of Points: 11000
MSE: 0.004773951373363102
RMSE: 0.06909378679275802
ARE: 10.28%
Number of Points: 12000
MSE: 0.002828980535079674
RMSE: 0.05318816160650483
ARE: 9.28%
Number of Points: 13000
MSE: 0.003053723481288594
RMSE: 0.05526050561919058
ARE: 9.34%
Number of Points: 14000
MSE: 0.003130027480057016
RMSE: 0.05594664851496482
ARE: 9.37%
Number of Points: 15000
MSE: 0.0029372190467879195
RMSE: 0.05419611652865839
ARE: 9.20%
Number of Points: 16000
MSE: 0.002900283

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import time

start_numpoints = 5000
end_numpoints = 50000
interval = 1000

results = []

for numpoints in range(start_numpoints, end_numpoints + 1, interval):
    start_time = time.time()
    print(f"Number of Points: {numpoints}")

    data = pd.read_csv(f'datasets/Energy/fuchs_v3_points_{numpoints}_noise_10.csv')
    features = data[['Intensity_(W_cm2)', 'Target_Thickness (um)', 'Focal_Distance_(um)']]
    target = data[['Max_Proton_Energy_(MeV)', 'Avg_Proton_Energy_(MeV)', 'Total_Proton_Energy_(MeV)']]

    # Split the dataset into train and test sets
    features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Define the hyperparameter grid to search
    param_grid = {
        'n_estimators': [400],
        'max_depth': [20],
        'min_samples_split': [2],
        'min_samples_leaf': [1],
        'max_features': ['auto'],
    }

    # Initialize the Random Forest Regressor
    rf = RandomForestRegressor(random_state=42)

    # Perform grid search to find the best hyperparameters
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(features_train, target_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Predict on the test set using the best model
    target_test_pred = best_model.predict(features_test)

    # Calculate the MSE, RMSE, and ARE for each target variable
    mse_errors = []
    rmse_errors = []
    are_errors = []
    for i, column in enumerate(target.columns):
        mse_error = mean_squared_error(target_test[column], target_test_pred[:, i])
        rmse_error = np.sqrt(mse_error)
        are_error = (mean_absolute_error(target_test[column], target_test_pred[:, i]) / np.mean(target_test[column])) * 100
        mse_errors.append(mse_error)
        rmse_errors.append(rmse_error)
        are_errors.append(are_error)

    # Store the results in a dictionary
    result = {
        'Number of Points': numpoints,
        'MSE Max_Proton_Energy': mse_errors[0],
        'RMSE Max_Proton_Energy': rmse_errors[0],
        'ARE Max_Proton_Energy': are_errors[0],
        'Elapsed Time (seconds)': time.time() - start_time
    }

    results.append(result)

    # Print the MSE, RMSE, and ARE for each target variable
    for i, column in enumerate(target.columns):
        print(f'MSE for {column}: {mse_errors[i]}')
        print(f'RMSE for {column}: {rmse_errors[i]}')
        print(f'ARE for {column}: {are_errors[i]}%')

    # Print the best hyperparameters from the grid search
    print("Best Hyperparameters:", grid_search.best_params_)

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('results.csv', index=False)


Number of Points: 5000
MSE for Max_Proton_Energy_(MeV): 0.005377386968566577
RMSE for Max_Proton_Energy_(MeV): 0.07333066867666337
ARE for Max_Proton_Energy_(MeV): 11.325847168873185%
MSE for Avg_Proton_Energy_(MeV): 0.00011814742043352185
RMSE for Avg_Proton_Energy_(MeV): 0.010869563948637584
ARE for Avg_Proton_Energy_(MeV): 10.372772401923475%
MSE for Total_Proton_Energy_(MeV): 1269779435039017.8
RMSE for Total_Proton_Energy_(MeV): 35633964.62701025
ARE for Total_Proton_Energy_(MeV): 13.65426929518398%
Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Number of Points: 6000
MSE for Max_Proton_Energy_(MeV): 0.0051640121878727355
RMSE for Max_Proton_Energy_(MeV): 0.07186106169458349
ARE for Max_Proton_Energy_(MeV): 10.946476882792904%
MSE for Avg_Proton_Energy_(MeV): 0.00011249036192735235
RMSE for Avg_Proton_Energy_(MeV): 0.010606147364964921
ARE for Avg_Proton_Energy_(MeV): 9.557057492320713%
MSE for To

MSE for Max_Proton_Energy_(MeV): 0.0038362063367734936
RMSE for Max_Proton_Energy_(MeV): 0.06193711598689023
ARE for Max_Proton_Energy_(MeV): 9.759259339067857%
MSE for Avg_Proton_Energy_(MeV): 0.00010592546246398794
RMSE for Avg_Proton_Energy_(MeV): 0.010292009641658325
ARE for Avg_Proton_Energy_(MeV): 9.281852175469583%
MSE for Total_Proton_Energy_(MeV): 632814658379947.6
RMSE for Total_Proton_Energy_(MeV): 25155807.6471408
ARE for Total_Proton_Energy_(MeV): 9.280389880061787%
Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Number of Points: 19000
MSE for Max_Proton_Energy_(MeV): 0.003445764654209759
RMSE for Max_Proton_Energy_(MeV): 0.05870063589272061
ARE for Max_Proton_Energy_(MeV): 9.606544396131762%
MSE for Avg_Proton_Energy_(MeV): 0.00010065630991423458
RMSE for Avg_Proton_Energy_(MeV): 0.01003276182884028
ARE for Avg_Proton_Energy_(MeV): 8.981401882375529%
MSE for Total_Proton_Energy_(MeV): 553

MSE for Max_Proton_Energy_(MeV): 0.003071319310234975
RMSE for Max_Proton_Energy_(MeV): 0.05541948493296356
ARE for Max_Proton_Energy_(MeV): 9.00143047115289%
MSE for Avg_Proton_Energy_(MeV): 8.947462271212888e-05
RMSE for Avg_Proton_Energy_(MeV): 0.009459102637783823
ARE for Avg_Proton_Energy_(MeV): 8.623564231022849%
MSE for Total_Proton_Energy_(MeV): 578984514350777.2
RMSE for Total_Proton_Energy_(MeV): 24062097.048070796
ARE for Total_Proton_Energy_(MeV): 8.83699271615081%
Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Number of Points: 32000
MSE for Max_Proton_Energy_(MeV): 0.0030946826659336065
RMSE for Max_Proton_Energy_(MeV): 0.05562987206468847
ARE for Max_Proton_Energy_(MeV): 9.220263971116434%
MSE for Avg_Proton_Energy_(MeV): 8.658938346791208e-05
RMSE for Avg_Proton_Energy_(MeV): 0.009305341663147683
ARE for Avg_Proton_Energy_(MeV): 8.667947295293683%
MSE for Total_Proton_Energy_(MeV): 5391

MSE for Max_Proton_Energy_(MeV): 0.0029310353084091518
RMSE for Max_Proton_Energy_(MeV): 0.0541390368256506
ARE for Max_Proton_Energy_(MeV): 8.726725154251564%
MSE for Avg_Proton_Energy_(MeV): 8.479992759414739e-05
RMSE for Avg_Proton_Energy_(MeV): 0.009208687615189658
ARE for Avg_Proton_Energy_(MeV): 8.466435563347295%
MSE for Total_Proton_Energy_(MeV): 556073459798785.6
RMSE for Total_Proton_Energy_(MeV): 23581209.88835784
ARE for Total_Proton_Energy_(MeV): 8.845050214910422%
Best Hyperparameters: {'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Number of Points: 45000
MSE for Max_Proton_Energy_(MeV): 0.0027331666707002445
RMSE for Max_Proton_Energy_(MeV): 0.05227969654368935
ARE for Max_Proton_Energy_(MeV): 8.666499573680905%
MSE for Avg_Proton_Energy_(MeV): 9.428383498036422e-05
RMSE for Avg_Proton_Energy_(MeV): 0.009709986353253243
ARE for Avg_Proton_Energy_(MeV): 8.692593186859005%
MSE for Total_Proton_Energy_(MeV): 637

In [3]:
pip install cuML


Defaulting to user installation because normal site-packages is not writeable
  ERROR: Command errored out with exit status 1:
   command: /apps/python/3.9-2022.05/bin/python -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/slurmtmp.17315713/pip-install-6wvkbnzx/cuml_6ff7b1039ec84063bff9f0c7db42d4f0/setup.py'"'"'; __file__='"'"'/tmp/slurmtmp.17315713/pip-install-6wvkbnzx/cuml_6ff7b1039ec84063bff9f0c7db42d4f0/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/slurmtmp.17315713/pip-wheel-pb0nvamb
       cwd: /tmp/slurmtmp.17315713/pip-install-6wvkbnzx/cuml_6ff7b1039ec84063bff9f0c7db42d4f0/
  Complete output (33 lines):
  running bdist_wheel
  running build
  /apps/python/3.9-2022.05/lib/python3.9/site-packages/setuptools/comma